In [1]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://m.realtytrac.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://m.realtytrac.com/mapsearch/sold/ca/')

True

# Thông tin nhóm

1612406 - Đặng Phương Nam

1612423 - Lê Minh nghĩa

# Câu hỏi

Cho các thông tin về căn nhà:

- Giá cho thuê trước khi bán.
- Địa chỉ.
- Loại nhà.
- Số phòng ngủ.
- Số nhà vệ sinh.
- Diện tích phần nhà.
- Diện tích toàn bộ lô đất.
- Năm xây dựng.
- Tiền thuế.
- Gần mấy trường học.
- Giá được bán gần đây nhất.

Hỏi giá trị của căn nhà là bao nhiêu tiền?

# Lợi ích khi trả lời được câu hỏi

Nhờ vào thông tin của căn nhà:

- Người bán có thể dự đoán được giá trị căn nhà mà mình muốn bán.
- Người mua có thể ước lượng được căn nhà mình muốn mua có giá cả hợp lý hay không?.
- Dự đoán được giá trị căn nhà của mình.
- ...

Dữ liệu thu thập từ trang web https://www.realtytrac.com/. Ta chỉ thu thập dữ liệu "các estate đã được bán tại California", ta có HTML cần parse: https://m.realtytrac.com/mapsearch/sold/ca/

### Start

In [2]:
# Import các thư viện cần thiết
import urllib.robotparser
import json
import os.path
import time
import traceback
import re
from datetime import datetime
from pprint import pprint

from requests_html import HTMLSession

In [3]:
session = HTMLSession()
r = session.get('https://m.realtytrac.com/mapsearch/sold/ca/')

# Get list url Countries of California
base_url = "https://m.realtytrac.com"
list_url = []

counties = r.html.find("option")
for county in counties:
    tail_url = county.attrs['value']
    if (tail_url):
        url = base_url + tail_url
        list_url.append(url)

list_url

['https://m.realtytrac.com/mapsearch/sold/ca/alameda-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/alpine-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/amador-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/butte-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/calaveras-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/colusa-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/contra-costa-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/del-norte-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/el-dorado-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/fresno-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/glenn-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/humboldt-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/imperial-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/inyo-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/kern-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/kings-count

In [4]:
# Hàm trả về generator, generator này trả về trang tiếp theo sau mỗi lần duyệt
def urlGenerator(baseUrl, startPage=1):
    i = startPage
    if i <= 1:
        yield baseUrl
        i = 2
    while True:
        yield f"{baseUrl}/p-{i}"
        i += 1
        
# Hàm parse để lấy thông tin chi tiết của estate từ detail Url của estate đã bán.      
def parseDetailPage(session: HTMLSession, url: str):
    #global logger
    #logger.info(f"Processing detail url: {url}")
    
    # Lấy mã HTML của trang web
    r = session.get(url)
    
    # Dictionary lưu lại kết quả
    result = {}
    
    # Lấy addrass của estate
    address = {}
    street = r.html.find("[itemprop = 'streetAddress']", first=True)
    if street is not None:
        street = street.text
        
    locality = r.html.find("[itemprop = 'addressLocality']", first=True)
    if locality is not None:
        locality = locality.text
        
    region = r.html.find("[itemprop = 'addressRegion']", first=True)
    if region is not None:
        region = region.text
    
    code = r.html.find("[itemprop = 'postalCode']", first=True)
    if code is not None:
        code = code.text
    
    address["street"] = street
    address["locality"] = locality
    address["region"] = region
    address["code"] = code
    
    result["address"] = address
    
    # Lấy thông tin ngày đã bán estate
    date_sold = r.html.find(".recently-sold", first=True)
    if date_sold is not None:
        result["date_sold"] = date_sold.text.replace("SOLD ON ", "")
    else:
        result["date_sold"] = None
    
    
    # Lấy giá cho thuê trước khi bán
    mortgage = r.html.find(".property-price-info", first=True)
    if mortgage is not None:
        result["mortgage"] = mortgage.text.replace("Est. Mortgage: ","")
    else:
        result["mortgage"] = None
    
    
    details = []
    detailTexts = r.html.find(".detail-text")
    if detailTexts is not None:
        for detailText in detailTexts:
            details.append(detailText.html)
    result["details"] = details
    
    # Lấy property info của estate
    propertyInfo = r.html.find(".property-info", first=True)
    if propertyInfo is not None:
        info = {}
        items = propertyInfo.find(".item")
        if items is not None:
            for item in items:
                name = item.find(".name", first=True).text.lower().replace(" ", "_")
                value = item.find(".value", first=True).text
                info[name] = value

        result["info"] = info
    else:
        result["info"] = None

    property_taxes = r.html.find(".property-taxes")
    if property_taxes is not None:
        for tax in property_taxes:
            key = tax.find(".section-head", first=True).text.lower().replace(" ", "_")
            taxes = {}
            items = tax.find(".item")
            if items is not None:
                for item in items:
                    name = item.find(".name", first=True).text.lower().replace(" ", "_")
                    value = item.find(".value", first=True).text
                    taxes[name] = value

            result[key] = taxes
  
    # Lấy số lượng trường học gần đó
    local_school = r.html.find(".property-schools", first=True)
    num_school = 0
    if local_school is not None:
        num_school = len(local_school.find(".schoolInfo"))
        
    result["school"] = num_school
    
    # Lấy thông tin tội phạm trong vùng
    crimes = {}
    local_crime_index = r.html.find(".property-local-crime", first=True)
    if local_crime_index is not None:
        type_crimes = local_crime_index.find(".gradeTitle")
        if type_crimes is not None:
            for type_crime in type_crimes:
                match = re.match(r"(.+?) = (\d+)", type_crime.text)
                if match is not None:
                    key = match.group(1).lower().replace(' ', '_')
                    value = int(match.group(2))
                    crimes[key] = value
                    
    result["local_crime_index"] = crimes

    # Lấy số lượng nhà bị tịch thu gần đó
    nearby_foreclosures = r.html.find(".property-nearby-fc", first=True)
    num_fc = 0
    if nearby_foreclosures is not None:
        num_fc = len(nearby_foreclosures.find(".nearby-property"))

    result["foreclosures"] = num_fc
    
    return result


In [5]:
def parseSearchPage(session, url):
    r = session.get(url)

    detailUrls = [link for link in r.html.absolute_links if 'property' in link]
    
    results = []
    for detailUrl in detailUrls:
        if not rp.can_fetch('*', detailUrl):
            print(f"SKIP: {detailUrl}")
            continue
        print(f"Parsing detail url {detailUrl}")
        result = parseDetailPage(session, detailUrl)
        results.append(result)
        
    #nextPageAnchor = r.html.find(".current + .page", first=True)
    nextPageAnchor = r.html.find(".current + .page", first=True)
    return results, nextPageAnchor is not None

In [6]:
def getAllCountyProperties(session, countyUrl):
    if countyUrl.endswith("/"):
        countyUrl = countyUrl[:-1]
    fileName = f"data/{os.path.basename(countyUrl)}.json"
    
    if os.path.exists(fileName):
        print(f"FILE EXISTS: {fileName}")
        return
    
    with open(fileName, "a") as fOut:
        for url in urlGenerator(countyUrl, 455):
            if not rp.can_fetch('*', url):
                print(f"SKIP: {url}")
                continue
            print(f"Parsing {url}")
            results, nextPage = parseSearchPage(session, url)
            for result in results:
                json.dump(result, fOut, ensure_ascii=False)
                fOut.write("\n")
            if not nextPage:
                break
                
            time.sleep(2)
        

In [7]:
#url = "https://m.realtytrac.com/mapsearch/sold/ca/alameda-county"
url = "https://m.realtytrac.com/mapsearch/sold/ca/orange-county"
session = HTMLSession()
getAllCountyProperties(session, url)

Parsing https://m.realtytrac.com/mapsearch/sold/ca/orange-county/p-455
Parsing detail url https://m.realtytrac.com/property/ca/irvine/92606/3572-redwood-st/156848355/
Parsing detail url https://m.realtytrac.com/property/ca/newport-beach/92662/1502-s-bay-front/157716838/
Parsing detail url https://m.realtytrac.com/property/ca/yorba-linda/92886/19063-rockwood-dr-18/16644640/
Parsing detail url https://m.realtytrac.com/property/ca/santa-ana/92705/2231-pavillion-dr/141950098/
Parsing detail url https://m.realtytrac.com/property/ca/anaheim/92806/2201-e-winston-rd/157133222/
Parsing detail url https://m.realtytrac.com/property/ca/mission-viejo/92692/28910-paseo-sabatini/49145495/
Parsing detail url https://m.realtytrac.com/property/ca/huntington-beach/92646/8201-racepoint-dr-105/16110103/
Parsing detail url https://m.realtytrac.com/property/ca/stanton/90680/7362-cerritos-ave-7/18621759/
Parsing detail url https://m.realtytrac.com/property/ca/mission-viejo/92694/2-parliament-pl/8565375/
Parsi

Parsing detail url https://m.realtytrac.com/property/ca/orange/92866/308-s-california-st/157762869/
Parsing detail url https://m.realtytrac.com/property/ca/fullerton/92831/1612-central-ave/140073099/
Parsing detail url https://m.realtytrac.com/property/ca/garden-grove/92843/11717-garden-grove-blvd/21806374/
Parsing detail url https://m.realtytrac.com/property/ca/garden-grove/92840/11402-fredrick-dr/157726672/
Parsing detail url https://m.realtytrac.com/property/ca/buena-park/90620/6411-morgan-way/3610093/
Parsing detail url https://m.realtytrac.com/property/ca/tustin/92782/2212-palermo/40438564/
Parsing detail url https://m.realtytrac.com/property/ca/anaheim/92801/1800-w-gramercy-ave-apt-60/40311435/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-beach/92651/753-manzanita-dr/155809324/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-beach/92651/445-arroyo-chico/158168463/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-niguel/92677/59-siena

Parsing detail url https://m.realtytrac.com/property/ca/huntington-beach/92646/8885-plumas-cir-1119b/156882030/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-niguel/92677/22932-via-cruz/155814773/
Parsing detail url https://m.realtytrac.com/property/ca/irvine/92606/1-gladstone-112/20608481/
Parsing detail url https://m.realtytrac.com/property/ca/fullerton/92835/413-thunderbird-ct/155777563/
Parsing detail url https://m.realtytrac.com/property/ca/tustin/92780/1026-colonial-way/40312779/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-hills/92653/24761-mendocino-ct/24738010/
Parsing detail url https://m.realtytrac.com/property/ca/huntington-beach/92646/20371-bluffside-cir-apt-402/40383967/
Parsing detail url https://m.realtytrac.com/property/ca/brea/92821/2351-stony-ln/155783067/
Parsing detail url https://m.realtytrac.com/property/ca/fullerton/92835/501-lakeside-dr/155747048/
Parsing detail url https://m.realtytrac.com/property/ca/foothill-ranch/92610/6-t

Parsing detail url https://m.realtytrac.com/property/ca/orange/92869/220-s-calle-grande/149358380/
Parsing detail url https://m.realtytrac.com/property/ca/fullerton/92831/2006-associated-rd-5/18398569/
Parsing detail url https://m.realtytrac.com/property/ca/costa-mesa/92627/283-23rd-st/157778212/
Parsing detail url https://m.realtytrac.com/property/ca/fullerton/92835/2030-domingo-rd/145488970/
Parsing detail url https://m.realtytrac.com/property/ca/costa-mesa/92627/2174-harmony-way/249320707/
Parsing detail url https://m.realtytrac.com/property/ca/capistrano-beach/92624/35651-beach-rd/29262141/
Parsing detail url https://m.realtytrac.com/property/ca/placentia/92870/314-sarah-ave/23767582/
Parsing detail url https://m.realtytrac.com/property/ca/santa-ana/92705/2526-n-linwood-ave/156847459/
Parsing https://m.realtytrac.com/mapsearch/sold/ca/orange-county/p-474
Parsing detail url https://m.realtytrac.com/property/ca/laguna-niguel/92677/4-shelter-cv/52133814/
Parsing detail url https://m.r

Parsing detail url https://m.realtytrac.com/property/ca/laguna-beach/92651/31372-holly-dr/157725911/
Parsing detail url https://m.realtytrac.com/property/ca/tustin/92782/2423-paseo-circulo/144775393/
Parsing https://m.realtytrac.com/mapsearch/sold/ca/orange-county/p-480
Parsing detail url https://m.realtytrac.com/property/ca/brea/92821/2101-arbor-cir/46032340/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-beach/92677/24805-eaton-ln/140794461/
Parsing detail url https://m.realtytrac.com/property/ca/mission-viejo/92691/26392-via-gaviota/51929436/
Parsing detail url https://m.realtytrac.com/property/ca/anaheim/92804/3127-w-graciosa-ln/19544200/
Parsing detail url https://m.realtytrac.com/property/ca/anaheim/92801/3311-w-lincoln-ave-apt-6/14698459/
Parsing detail url https://m.realtytrac.com/property/ca/costa-mesa/92627/2390-westminster-ave/145892865/
Parsing detail url https://m.realtytrac.com/property/ca/irvine/92602/64-copper-mine/251899209/
Parsing detail url https://m

...

Parsing detail url https://m.realtytrac.com/property/ca/tustin/92782/15207-carrington-way/206577957/
Parsing detail url https://m.realtytrac.com/property/ca/orange/92869/129-n-cross-creek-rd/147189315/
Parsing detail url https://m.realtytrac.com/property/ca/los-alamitos/90720/10761-walnut-st/147490495/
Parsing detail url https://m.realtytrac.com/property/ca/villa-park/92861/17962-aberdeen-ln/51717223/
Parsing detail url https://m.realtytrac.com/property/ca/la-habra/90631/1210-w-lambert-rd-51/40400254/
Parsing detail url https://m.realtytrac.com/property/ca/huntington-beach/92649/5831-middlecoff-dr/10093970/
Parsing detail url https://m.realtytrac.com/property/ca/rancho-santa-margarita/92688/15-vista-mesa-38/157813604/
Parsing detail url https://m.realtytrac.com/property/ca/la-habra/90631/1571-dorwood-ave/157715231/
Parsing detail url https://m.realtytrac.com/property/ca/garden-grove/92843/11841-stuart-dr/157168349/
Parsing detail url https://m.realtytrac.com/property/ca/ladera-ranch/92

Parsing detail url https://m.realtytrac.com/property/ca/placentia/92870/302-eagle-dr/146353504/
Parsing detail url https://m.realtytrac.com/property/ca/la-habra-heights/90631/2255-canonita-dr/22260059/
Parsing detail url https://m.realtytrac.com/property/ca/mission-viejo/92692/23471-villena/23327743/
Parsing detail url https://m.realtytrac.com/property/ca/irvine/92620/203-parkwood/327409983/
Parsing detail url https://m.realtytrac.com/property/ca/santa-ana/92701/1510-eastside-ave/19630204/
Parsing detail url https://m.realtytrac.com/property/ca/mission-viejo/92692/22181-brookpine/19877361/
Parsing detail url https://m.realtytrac.com/property/ca/garden-grove/92841/12811-josephine-st/21985708/
Parsing detail url https://m.realtytrac.com/property/ca/costa-mesa/92627/353-e-18th-st-c/325968797/
Parsing https://m.realtytrac.com/mapsearch/sold/ca/orange-county/p-736
Parsing detail url https://m.realtytrac.com/property/ca/fullerton/92833/3073-primrose-ln/19593672/
Parsing detail url https://m.

Parsing detail url https://m.realtytrac.com/property/ca/san-clemente/92673/600-corte-calmo/44010819/
Parsing detail url https://m.realtytrac.com/property/ca/anaheim/92807/6523-e-camino-vis-6/8630302/
Parsing https://m.realtytrac.com/mapsearch/sold/ca/orange-county/p-742
Parsing detail url https://m.realtytrac.com/property/ca/huntington-beach/92646/8312-kiner-dr/157740664/
Parsing detail url https://m.realtytrac.com/property/ca/fullerton/92833/4207-w-ash-ave/40361034/
Parsing detail url https://m.realtytrac.com/property/ca/rancho-santa-margarita/92688/13-silktassel-149/15763230/
Parsing detail url https://m.realtytrac.com/property/ca/irvine/92614/12-comiso/44056638/
Parsing detail url https://m.realtytrac.com/property/ca/costa-mesa/92627/2435-orange-ave-apt-a1/157769243/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-niguel/92677/24871-hamlet-way/40349633/
Parsing detail url https://m.realtytrac.com/property/ca/santa-ana/92707/3001-s-sycamore-st-apt-h/27331483/
Parsing d

Parsing detail url https://m.realtytrac.com/property/ca/seal-beach/90740/705-beachcomber-dr/25114894/
Parsing detail url https://m.realtytrac.com/property/ca/santa-ana/92705/1212-castlegate-ln/158196352/
Parsing detail url https://m.realtytrac.com/property/ca/irvine/92620/12-brena/156855660/
Parsing detail url https://m.realtytrac.com/property/ca/newport-beach/92660/2-weybridge-ct/157773335/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-hills/92656/26501-merienda-apt-7/27412404/
Parsing detail url https://m.realtytrac.com/property/ca/santa-ana/92701/711-eastside-ave/30615521/
Parsing detail url https://m.realtytrac.com/property/ca/newport-beach/92663/11-bluefin-ct/17878090/
Parsing detail url https://m.realtytrac.com/property/ca/la-habra/90631/2051-e-whittier-blvd/18182565/
Parsing detail url https://m.realtytrac.com/property/ca/orange/92868/264-n-jewell-pl/16842472/
Parsing detail url https://m.realtytrac.com/property/ca/laguna-niguel/92677/24363-el-pilar-130/19282571

Parsing detail url https://m.realtytrac.com/property/ca/ladera-ranch/92694/11-bramford-st/23244453/
Parsing detail url https://m.realtytrac.com/property/ca/san-clemente/92673/2812-via-blanco/157786580/
Parsing detail url https://m.realtytrac.com/property/ca/stanton/90680/10251-lotus-ct/249330770/
Parsing detail url https://m.realtytrac.com/property/ca/fullerton/92831/700-maertin-ln/52506331/
Parsing detail url https://m.realtytrac.com/property/ca/anaheim/92805/1804-e-willow-st/22548978/
Parsing detail url https://m.realtytrac.com/property/ca/placentia/92870/2102-rosemont-st/12840371/
Parsing detail url https://m.realtytrac.com/property/ca/garden-grove/92840/12836-palm-st-7/208271726/
Parsing detail url https://m.realtytrac.com/property/ca/newport-beach/92660/611-vista-bonita/157769396/
Parsing detail url https://m.realtytrac.com/property/ca/irvine/92618/156-allium/239791628/
Parsing detail url https://m.realtytrac.com/property/ca/san-clemente/92672/227-avenida-monterey/40447758/
Parsin

In [39]:
import json
import os.path

def xstr(s):
    return '' if s is None else str(s)

def preprocess(inputFile : str, outputFile : str):
    if os.path.exists(outputFile):
        print(f"FILE EXISTS: {outputFile}")
        return
    
    if not os.path.exists(inputFile):
        print(f"Input file not found: {inputFile}")
        return
    
    with open(inputFile) as fIn, open(outputFile, "w") as fOut:
        tab = "\t"
        headers = (
            "address_street",
            "address_locality",
            "address_region",
            "address_code",
            "date_sold",
            "mortgage",
            "info_type",
            "info_bedrooms",
            "info_bathrooms",
            "info_size",
            "info_lot_size",
            "info_year_built",
            "info_est_value",
            "info_sold_price",
            "info_property_id",
            "info_county",
            "info_parcel_number",
            "taxes_land",
            "taxes_improvements",
            "taxes_total",
            "taxes_taxes",
            "school",
            "total_crime",
            "violent_crime",
            "property_crime",
            "foreclosures",
        )
        
        # Ghi các tên cột vào file output, mỗi cột ngăn cách bởi "\t"
        fOut.write(f"{tab.join(headers)}\n")
        
        for line in fIn:
            row = json.loads(line)
            
            # Lấy địa chỉ
            address = row.get("address")
            address_street = address.get("street")
            address_locality = address.get("locality")
            address_region = address.get("region")
            address_code = address.get("code")
            
            # Lấy ngày bán thành công
            date_sold = row.get("date_sold")
            
            # Lấy giá cho thuê mỗi tháng (trước khi được bán)
            mortgage = row.get("mortgage")
            if mortgage is not None:
                mortgage = float(mortgage.replace("$","").replace(",","").replace("/mo","").replace("Est. Refinance: ", ""))
            
            # Lấy phần info
            info = row.get("info")
            
            # Type
            info_type = info.get("type")
            
            # Bedrooms
            info_bedrooms = info.get("bedrooms")
            if info_bedrooms  == "Contact Agent":
                info_bedrooms = None
            else:
                info_bedrooms = float(info_bedrooms)
            
            # Bathrooms
            info_bathrooms = info.get("bathrooms")
            if info_bathrooms == "Contact Agent":
                info_bathrooms = None
            else:
                info_bathrooms = float(info_bathrooms)
            
            # Size
            info_size = info.get("size")
            if info_size  == "Contact Agent":
                info_size = None
            else:
                info_size = float(info_size.replace(",","").replace(" sqft", ""))
            
            # Lot size
            info_lot_size = info.get("lot_size")
            if info_lot_size  == "Contact Agent":
                info_lot_size = None
            else:
                info_lot_size = float(info_lot_size.replace(",","").replace(" sqft", "").replace(" acres", ""))
            
            # Year build
            info_year_built = info.get("year_built")
            if info_year_built  == "Contact Agent":
                info_year_built = None
            else:
                info_year_built = int(info_year_built)
        
            # Est value
            info_est_value = info.get("est._value")
            if info_est_value is not None:
                info_est_value = float(info_est_value.replace("$","").replace(",",""))
            
            # Sold price
            info_sold_price = info.get("sold_price")
            if info_sold_price is not None:
                if info_sold_price != "N/A":
                    info_sold_price = float(info_sold_price.replace("$","").replace(",",""))
            
            # ID
            info_property_id = info.get("property_id")
            
            # County
            info_county = info.get("county")
            
            # Parcel_number
            info_parcel_number = info.get("parcel_number")
            
            # Lấy phần Taxes
            taxes = row.get("property_taxes")
            taxes_land = None
            taxes_improvements = None
            taxes_total = None
            taxes_taxes = None
            if taxes is not None:
                taxes_land = float(taxes.get("land").replace("$","").replace(",",""))
                taxes_improvements = float(taxes.get("improvements").replace("$","").replace(",",""))
                taxes_total = float(taxes.get("total").replace("$","").replace(",",""))
                taxes_taxes = taxes.get("taxes").replace("$","").replace(",","")
           
            # Lấy số lượng scholl gần đó
            school = int(row.get("school"))
            
            # Lấy tình hình tội phạm
            crime = row.get("local_crime_index")
            if crime is not None:
                total_crime = crime.get("total_crime")
                if total_crime is not None:
                    total_crime = int(total_crime)
                violent_crime = crime.get("violent_crime")
                if violent_crime is not None:
                    violent_crime = int(violent_crime)
                property_crime = crime.get("property_crime")
                if property_crime is not None:
                    property_crime = int(property_crime)
                
            
            # Lấy số lượng foreclosures gần đó
            foreclosures = int(row.get("foreclosures"))
        
        
            row = (
                xstr(address_street),
                xstr(address_locality),
                xstr(address_region),
                xstr(address_code),
                xstr(date_sold),
                xstr(mortgage),
                xstr(info_type),
                xstr(info_bedrooms),
                xstr(info_bathrooms),
                xstr(info_size),
                xstr(info_lot_size),
                xstr(info_year_built),
                xstr(info_est_value),
                xstr(info_sold_price),
                xstr(info_property_id),
                xstr(info_county),
                xstr(info_parcel_number),
                xstr(taxes_land),
                xstr(taxes_improvements),
                xstr(taxes_total),
                xstr(taxes_taxes),
                xstr(school),
                xstr(total_crime),
                xstr(violent_crime),
                xstr(property_crime),
                xstr(foreclosures),
            )
            
            # Ghi xuống file output
            fOut.write(f"{tab.join(row)}\n")

In [40]:
inputFile = "data/full-orange-county.json"
outputFile = "data/orange-county.csv"
preprocess(inputFile, outputFile)

In [41]:
import pandas as pd

In [42]:
file_csv = "data/orange-county.csv"
data_df =  pd.read_csv(file_csv, sep='\t')
data_df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,93496030,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4
1,1013 N Mountain View Pl,Fullerton,CA,92831,12/01/1993,3289.0,Single Family Residence,4.0,2.0,2037.0,...,2954406,208547.0,130222.0,338769.0,3944 (1.16 %),3,65.0,49.0,100.0,4
2,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,78623129,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4
3,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,15512543,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4
4,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,93785045,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4
5,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,29247108,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4
6,365 W Skyline Dr,La Habra Heights,CA,90631,05/25/1965,182.0,Single Family Residence,2.0,2.0,1714.0,...,8240004014,56212.0,50609.0,106821.0,2082 (1.94 %),3,35.0,17.0,79.0,4
7,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,50314226,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4
8,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,52719141,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4
9,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,26251319,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4


In [43]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21268 entries, 0 to 21267
Data columns (total 26 columns):
address_street        21204 non-null object
address_locality      21266 non-null object
address_region        21268 non-null object
address_code          21268 non-null int64
date_sold             21268 non-null object
mortgage              21134 non-null float64
info_type             21268 non-null object
info_bedrooms         19148 non-null float64
info_bathrooms        19159 non-null float64
info_size             20282 non-null float64
info_lot_size         14829 non-null float64
info_year_built       19737 non-null float64
info_est_value        1297 non-null float64
info_sold_price       19837 non-null float64
info_property_id      21268 non-null int64
info_county           21268 non-null object
info_parcel_number    21268 non-null int64
taxes_land            20981 non-null float64
taxes_improvements    20981 non-null float64
taxes_total           20981 non-null float64
taxe

In [44]:
unique_data_df = data_df.drop_duplicates()

In [45]:
unique_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18893 entries, 0 to 21267
Data columns (total 26 columns):
address_street        18841 non-null object
address_locality      18891 non-null object
address_region        18893 non-null object
address_code          18893 non-null int64
date_sold             18893 non-null object
mortgage              18772 non-null float64
info_type             18893 non-null object
info_bedrooms         16966 non-null float64
info_bathrooms        16976 non-null float64
info_size             17993 non-null float64
info_lot_size         13186 non-null float64
info_year_built       17505 non-null float64
info_est_value        1139 non-null float64
info_sold_price       17633 non-null float64
info_property_id      18893 non-null int64
info_county           18893 non-null object
info_parcel_number    18893 non-null int64
taxes_land            18625 non-null float64
taxes_improvements    18625 non-null float64
taxes_total           18625 non-null float64
taxe

In [54]:
unique_data_df.to_csv("data/delete-duplicate-orange.csv", sep='\t', index=False)

In [56]:
df =  pd.read_csv("data/delete-duplicate-orange.csv", sep='\t')
df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,93496030,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4
1,1013 N Mountain View Pl,Fullerton,CA,92831,12/01/1993,3289.0,Single Family Residence,4.0,2.0,2037.0,...,2954406,208547.0,130222.0,338769.0,3944 (1.16 %),3,65.0,49.0,100.0,4
2,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,78623129,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4
3,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,15512543,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4
4,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,93785045,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4
5,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,29247108,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4
6,365 W Skyline Dr,La Habra Heights,CA,90631,05/25/1965,182.0,Single Family Residence,2.0,2.0,1714.0,...,8240004014,56212.0,50609.0,106821.0,2082 (1.94 %),3,35.0,17.0,79.0,4
7,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,50314226,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4
8,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,52719141,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4
9,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,26251319,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18893 entries, 0 to 18892
Data columns (total 26 columns):
address_street        18841 non-null object
address_locality      18891 non-null object
address_region        18893 non-null object
address_code          18893 non-null int64
date_sold             18893 non-null object
mortgage              18772 non-null float64
info_type             18893 non-null object
info_bedrooms         16966 non-null float64
info_bathrooms        16976 non-null float64
info_size             17993 non-null float64
info_lot_size         13186 non-null float64
info_year_built       17505 non-null float64
info_est_value        1139 non-null float64
info_sold_price       17633 non-null float64
info_property_id      18893 non-null int64
info_county           18893 non-null object
info_parcel_number    18893 non-null int64
taxes_land            18625 non-null float64
taxes_improvements    18625 non-null float64
taxes_total           18625 non-null float64
taxe

In [7]:
inputFiles = ["data/orange-county-1.json", "data/orange-county-2.json", "data/orange-county-3.json", "data/orange-county-4.json"]
outputFile = "data/full-orange-county.json"

with open(outputFile, "a+") as fOut:
    for inputFile in inputFiles:
        with open(inputFile, "r") as fIn:
            data = fIn.read()
            fOut.write(data)

### End